In [5]:
import xarray as xr
import numpy as np
import os
from datetime import datetime

ModuleNotFoundError: No module named 'utils'

In [10]:
os.listdir('../reforecast_v12/slp/mean_hours')[0][-18:-8]

'2012021500'

In [2]:
era_mslp = xr.open_dataset('../era5/mslp.nc',chunks={'time':100,'latitude':100,'longitude':100})

In [3]:
era_mslp

,Array,Chunk
Bytes,20.05 GB,4.00 MB
Shape,"(14440, 241, 1440)","(100, 100, 100)"
Count,6526 Tasks,6525 Chunks
Type,float32,numpy.ndarray


In [8]:
def dt2cal(dt):
    """
    Convert array of datetime64 to a calendar array of year, month, day, hour,
    minute, seconds, microsecond with these quantites indexed on the last axis.

    Parameters
    ----------
    dt : datetime64 array (...)
        numpy.ndarray of datetimes of arbitrary shape

    Returns
    -------
    cal : uint32 array (..., 7)
        calendar array with last axis representing year, month, day, hour,
        minute, second, microsecond
    """

    # allocate output 
    out = np.empty(dt.shape + (7,), dtype="u4")
    # decompose calendar floors
    Y, M, D, h, m, s = [dt.astype(f"M8[{x}]") for x in "YMDhms"]
    out[..., 0] = Y + 1970 # Gregorian Year
    out[..., 1] = (M - Y) + 1 # month
    out[..., 2] = (D - M) + 1 # dat
    out[..., 3] = (dt - D).astype("m8[h]") # hour
    out[..., 4] = (dt - h).astype("m8[m]") # minute
    out[..., 5] = (dt - m).astype("m8[s]") # second
    out[..., 6] = (dt - s).astype("m8[us]") # microsecond
    return out
def replace_year(x, year):
    """ Year must be a leap year for this to work """
    # Add number of days x is from JAN-01 to year-01-01 
    x_year = np.datetime64(str(year)+'-01-01') +  (x - x.astype('M8[Y]'))

    # Due to leap years calculate offset of 1 day for those days in non-leap year
    yr_mn = x.astype('M8[Y]') + np.timedelta64(59,'D')
    leap_day_offset = (yr_mn.astype('M8[M]') - yr_mn.astype('M8[Y]') - 1).astype(np.int)

    # However, due to days in non-leap years prior March-01, 
    # correct for previous step by removing an extra day
    non_leap_yr_beforeMarch1 = (x.astype('M8[D]') - x.astype('M8[Y]')).astype(np.int) < 59
    non_leap_yr_beforeMarch1 = np.logical_and(non_leap_yr_beforeMarch1, leap_day_offset).astype(np.int)
    day_offset = np.datetime64('1970') - (leap_day_offset - non_leap_yr_beforeMarch1).astype('M8[D]')

    # Finally, apply the day offset 
    x_year = x_year - day_offset
    return x_year

In [13]:

files = os.listdir('../reforecast_v12/slp/mean_hours')
file_dt = [datetime.strptime(n[-18:-8],'%Y%m%d%H') for n in files]
djf = [n for n in file_dt if n.month in [12,1,2] and n.year in np.arange(2007,2014.1,1)]
nov = [n for n in file_dt if n.month == 11 and n.year in np.arange(2007,2014.1,1)]
nov = [n for n in nov if n.day <= 23]
djfnov = djf + nov
files_ndjf = [n.strftime('../reforecast_v12/slp/mean_hours/nh_time_pres_msl_%Y%m%d%H_mean.nc') for n in djfnov]
date = '2018-01-01'
d64 = np.datetime64(date,'D')
date_range = replace_year(np.arange(d64-10,d64+11), 2012)
dt2 = dt2cal(date_range)
files = os.listdir(f'../reforecast_v12/slp/mean_hours')
file_dt = [datetime.strptime(n[-18:-8],'%Y%m%d%H') for n in files]
date_tuple_list = [(dt2[n,1],dt2[n,2]) for n in range(len(dt2))]
file_list = [n for n in file_dt if (n.month, n.day) in date_tuple_list]

v12_stat = 'mean'
files_ndjf = [n.strftime(f'../reforecast_v12/slp/{v12_stat}_hours/nh_time_pres_msl_%Y%m%d%H_{v12_stat}.nc') for n in file_list]

In [14]:
files_ndjf

['../reforecast_v12/slp/mean_hours/nh_time_pres_msl_2011123100_mean.nc',
 '../reforecast_v12/slp/mean_hours/nh_time_pres_msl_2014122600_mean.nc',
 '../reforecast_v12/slp/mean_hours/nh_time_pres_msl_2019010500_mean.nc',
 '../reforecast_v12/slp/mean_hours/nh_time_pres_msl_2001122300_mean.nc',
 '../reforecast_v12/slp/mean_hours/nh_time_pres_msl_2008010700_mean.nc',
 '../reforecast_v12/slp/mean_hours/nh_time_pres_msl_2016010900_mean.nc',
 '../reforecast_v12/slp/mean_hours/nh_time_pres_msl_2011122700_mean.nc',
 '../reforecast_v12/slp/mean_hours/nh_time_pres_msl_2003122500_mean.nc',
 '../reforecast_v12/slp/mean_hours/nh_time_pres_msl_2017011100_mean.nc',
 '../reforecast_v12/slp/mean_hours/nh_time_pres_msl_2012122200_mean.nc',
 '../reforecast_v12/slp/mean_hours/nh_time_pres_msl_2016010600_mean.nc',
 '../reforecast_v12/slp/mean_hours/nh_time_pres_msl_2015010400_mean.nc',
 '../reforecast_v12/slp/mean_hours/nh_time_pres_msl_2003122900_mean.nc',
 '../reforecast_v12/slp/mean_hours/nh_time_pres_msl

In [15]:
gefs_mslp = xr.open_mfdataset(files_ndjf,
                              coords='minimal',
                              data_vars='minimal',
                              compat='override',
                              combine='nested',concat_dim='date',chunks={'date':1,'time':1})

In [17]:
gefs_mslp.load()

<xarray.Dataset>
Dimensions:    (date: 420, latitude: 241, longitude: 1440, time: 64)
Coordinates:
  * time       (time) float64 -21.0 -18.0 -15.0 -12.0 ... 162.0 165.0 168.0
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.25 359.5 359.75
  * latitude   (latitude) float32 60.0 59.75 59.5 59.25 ... 0.75 0.5 0.25 0.0
Dimensions without coordinates: date
Data variables:
    msl        (date, time, latitude, longitude) float32 nan nan ... 101231.914

In [6]:
gefs_mslp

,Array,Chunk
Bytes,195.90 GB,40.00 kB
Shape,"(2205, 64, 241, 1440)","(1, 1, 100, 100)"
Count,35817362 Tasks,6350400 Chunks
Type,float32,numpy.ndarray
